In [1]:
import requests
import pandas as pd
import pprint
import csv

#### Get geoJSON world cities polygons
Getting polygon lat-long bounding data for world cities from the json file linked below. This will be used for the geometry bounding polygons on the OpenStreetMap ohsome API calls for the bike lane and bike dock data. 

URL for geoJSON with geometry polygons for world cities: https://raw.githubusercontent.com/drei01/geojson-world-cities/master/cities.geojson

In [2]:
# url with geoJSON lat long polygons for major world cities
geoJSON_URL = 'https://raw.githubusercontent.com/drei01/geojson-world-cities/master/cities.geojson'

# extract data into json 
r = requests.get(geoJSON_URL)
data = r.json()

# convert to dataframe with nested keys
df = pd.json_normalize(data, record_path=['features'])

In [3]:
# only look at cities within the European continent (eg. there will be multiple cities named Paris throughout the world, only want the one in France)

# lat long "bounding box" to filter for subest of cities - europe specific
west_lat = -12
east_lat = 40
north_long = 72
south_long = 36

# check to see if first lat long pair is within the bounding box and append to new dataframe
europe_df =[]
for i in range(0,len(df)):
    if ((df['geometry.coordinates'][i][0][0][0] > west_lat) & (df['geometry.coordinates'][i][0][0][0] < east_lat) 
        & (df['geometry.coordinates'][i][0][0][1] > south_long) & (df['geometry.coordinates'][i][0][0][1] < north_long)): # looking at first lat long pair for filtering
        europe_df.append(df.iloc[i])

europe_df = pd.DataFrame(europe_df)
europe_df

,type,properties.NAME,geometry.type,geometry.coordinates
0,Feature,TORSHAVN,Polygon,"[[[-6.75864, 62.01517], [-6.76184, 62.01528], ..."
1,Feature,LERWICK,Polygon,"[[[-1.16446, 60.16395], [-1.16168, 60.16221], ..."
2,Feature,HONNINGSVAG,Polygon,"[[[26.00776, 70.97373], [25.99542, 70.97484], ..."
3,Feature,DYFJORD,Polygon,"[[[27.31813, 70.9571], [27.31875, 70.95818], [..."
4,Feature,HAMMERFEST,Polygon,"[[[23.68397, 70.67101], [23.68239, 70.6712], [..."
...,...,...,...,...
25468,Feature,ISKENDERUN,Polygon,"[[[36.18381, 36.58144], [36.18169, 36.57772], ..."
25477,Feature,HALAB (ALEPPO),Polygon,"[[[37.14989, 36.23594], [37.15144, 36.23325], ..."
25478,Feature,HATAY (ANTIOCH),Polygon,"[[[36.1665, 36.20008], [36.16483, 36.19961], [..."
25480,Feature,ASH SHAYKH SAID,Polygon,"[[[37.16472, 36.15975], [37.16455, 36.15978], ..."


In [4]:
# note - sometimes there are still multiple cities with the same name. However, the geometry coordinates are very similar. Assumed to be the same place and use the first entry in the dataframe
europe_df[europe_df['properties.NAME'] == 'LONDON']

,type,properties.NAME,geometry.type,geometry.coordinates
1116,Feature,LONDON,Polygon,"[[[0.0, 51.51039], [-0.00722, 51.51047], [-0.0..."
2818,Feature,LONDON,Polygon,"[[[0.0, 51.51039], [0.0, 51.6], [0.0, 51.64111..."
2839,Feature,LONDON,Polygon,"[[[0.06936, 51.62853], [0.07169, 51.62958], [0..."
2923,Feature,LONDON,Polygon,"[[[0.0, 51.36253], [0.0, 51.50683], [0.01417, ..."


In [5]:
# for each city we are analyzing, get lat long geometry polygon for ohsome API input

city_list = ['LONDON', 'AMSTERDAM', 'PARIS', 'BERLIN', 'MUNICH','BARCELONA', 'ROMA', 'LISBOA'] # list of cities in our analysis - some are in different languages/confirm spelling
city_geom = []

# for each city, get the geometry polygon and convert it to a string in the format for the ohsome API
for city in city_list: # loop through list of cities
    
    row = europe_df[europe_df['properties.NAME'] == city].index[0] # getting first row where the city name matches 
    city_geom_string = ''

    # for that city, loop through the coordinates column and convert to string
    for i in range(0,len(df['geometry.coordinates'][row][0])):
        s = ', '.join(str(x) for x in df['geometry.coordinates'][row][0][i])
        city_geom_string += s + ','
    
    city_geom.append(city_geom_string) # append geometry strings to list

# create dataframe of cities and their geometry strings
city_geom_df = pd.DataFrame({'Location': city_list, 'Geometry String': city_geom})
city_geom_df

,Location,Geometry String
0,LONDON,"0.0, 51.51039,-0.00722, 51.51047,-0.00811, 51...."
1,AMSTERDAM,"4.78211, 52.42692,4.78464, 52.42711,4.7927, 52..."
2,PARIS,"2.57778, 48.66292,2.57478, 48.666,2.55333, 48...."
3,BERLIN,"13.27198, 52.5758,13.27506, 52.57692,13.27872,..."
4,MUNICH,"11.61919, 48.16908,11.61814, 48.16745,11.62511..."
5,BARCELONA,"2.20506, 41.45139,2.20806, 41.44797,2.21339, 4..."
6,ROMA,"12.56719, 41.8973,12.57758, 41.89308,12.57778,..."
7,LISBOA,"-9.19264, 38.73531,-9.19936, 38.73539,-9.20383..."


#### API calls for ohsome OpenStreetMap bike lane and bike dock data

##### Bike Lane Length Extraction

In [6]:
# function for getting the total length of roadway in a city
def roadwayLength_extraction(city, geometry, groupByKey, time, filterKey='type:way'):
    '''Function that creates a dataframe with total length of roadway type
    city - city name
    geometry - boundary area from which to pull that data for
    filterKey - how to filter the results; can use multiple attributes; ex geometry type (type:way)
    grouByKey - how to seperate results from within the filter tag (ex. cycleway)
    time - the end point timestamp from when the data for OpenStreetMap is pulled (ex. 2010-01-01 will pull data existing at Jan 1 2010)
    Function specifically use for length extraction with a groupBy tag. Other API calls will have different URLs
    '''
    URL = 'https://api.ohsome.org/v1/elements/length/groupBy/tag'
    data = {"bpolys": geometry, "format": 'csv', "filter": filterKey, "groupByKey": groupByKey, "time": time}
    response = requests.post(URL, data=data)
    decoded_content = response.content.decode('utf-8')
    
    # create and format dataframe from csv file
    cr = csv.reader(decoded_content.splitlines(), delimiter=';')
    data_list = list(cr)
    df = pd.DataFrame({'Road Type': data_list[3], city: data_list[4]})
    df = df[1:] # remove timestamp row
    df[city] = df[city].astype(float) # convert length column to float
    
    return df

In [7]:
# create dataframe of summation of bike lanes, seperated by type, for each city

groupByKey = 'bicycle' # key for grouping roadway type
time = '2010-01-01' # will need to update this for each year we're pulling data

# make dataframe for one city as starter df for merge - figure out how to create df without doing this??
df_bike = roadwayLength_extraction('London', city_geom_df['Geometry String'][0], groupByKey, time) 

# iterate through list of cities
for index, row in city_geom_df.iterrows():
    city = row['Location']
    geom_str = row['Geometry String']
    df = roadwayLength_extraction(city, geom_str, groupByKey, time) # pull roadway data for specific city
    df_bike = pd.merge(df_bike, df, how='outer') # merge city data into one dataframe 

df_bike

,Road Type,London,LONDON,AMSTERDAM,PARIS,BERLIN,MUNICH,BARCELONA,ROMA,LISBOA
0,remainder,16593343.37,16593343.37,2016253.37,8683088.36,10637693.45,4335702.57,1573272.91,44999.22,1389895.52
1,bicycle=yes,147009.56,147009.56,10205.39,79815.04,426983.01,190654.95,8255.36,93.03,1052.29
2,bicycle=no,72175.36,72175.36,52444.55,3081.25,47867.38,116068.94,NaN,NaN,NaN
3,bicycle=designated,803.38,803.38,228.80,NaN,46778.47,80344.07,NaN,NaN,240.03
4,bicycle=dismount,632.62,632.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,bicycle=permissive,37358.67,37358.67,474.35,10955.03,7290.09,14512.46,NaN,NaN,NaN
6,bicycle=unknown,1615.54,1615.54,NaN,NaN,891.21,NaN,NaN,NaN,NaN
7,bicycle=opposite,774.40,774.40,NaN,NaN,NaN,1311.66,NaN,NaN,NaN
8,bicycle=opposite_lane,156.52,156.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,bicycle=true,NaN,NaN,NaN,392.24,NaN,NaN,NaN,NaN,NaN


In [8]:
# create dataframe of summation of cycle lanes, seperated by type, for each city
groupByKey = 'cycleway'
time = '2010-01-01' # will need to update this for each year we're pulling data

df_cycle = roadwayLength_extraction('London', city_geom_df['Geometry String'][0], groupByKey, time) # make dataframe for one city as starter df for merge

# iterate through list of cities
for index, row in city_geom_df.iterrows():
    city = row['Location']
    geom_str = row['Geometry String']
    df = roadwayLength_extraction(city, geom_str, groupByKey, time) # pull roadway data for specific city
    df_cycle = pd.merge(df_cycle, df, how='outer') # merge city data into one dataframe 

df_cycle

,Road Type,London,LONDON,AMSTERDAM,PARIS,BERLIN,MUNICH,BARCELONA,ROMA,LISBOA
0,remainder,16714933.12,16714933.12,1956805.91,8536313.07,10756573.18,4437578.81,1566662.59,45092.24,1391187.84
1,cycleway=lane,33299.60,33299.60,5686.06,57817.79,95349.79,11323.64,13386.94,NaN,NaN
2,cycleway=track,68172.88,68172.88,86526.25,95914.82,285357.76,273326.58,1222.97,NaN,NaN
3,cycleway=opposite,3862.70,3862.70,27808.99,14403.93,22219.80,12196.02,NaN,NaN,NaN
4,cycleway=shared,6846.57,6846.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cycleway=opposite_lane,14363.31,14363.31,1185.72,22638.61,NaN,1720.39,NaN,NaN,NaN
6,cycleway=yes,1340.41,1340.41,NaN,287.47,5205.69,3.97,NaN,NaN,NaN
7,cycleway=opposite_track,3864.01,3864.01,1593.54,7132.99,1166.24,1264.04,255.78,NaN,NaN
8,cycleway=segregated,7005.30,7005.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cycleway=lane_right;opposite_track,30.14,30.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Bike Lane length extraction - Filter
Based on detailed filter for bike/cycle ways, since they can be tagged differently in OpenStreetMap. 
The filter is based on Hochmair, Zielstra, and Neis’s paper "Assessing the completeness of bicycle trails and designated lane features in OpenStreetMap for the United States and Europe". 
In their study they explored the cycling features in the United States and Europe. The filter for this layer consists of their filter combination and is extended with other tags of cycling infrastructure mapping methods listed on the corresponding OSM wiki pages.

Filter is taken from here: 
https://hex.ohsome.org/#/cycleways_w/2020-06-01T00:00:00Z/4/50.28251213872028/16.31249248981477

In [9]:
# filterKey = 'type:way and sidewalk:left:bicycle=yes or cycleway:left=shared_lane or cyclestreet=yes or cycleway:left=shared_busway or cycleway:right=shared_busway or cycleway=shared_busway 
# or cycleway=opposite_lane or highway=bridleway and bicycle=no or highway=track and bicycle=designated and motor_vehicle=no or bicycle=use_sidepath or cycleway=opposite and oneway:bicycle=no 
# or sidewalk:right:bicycle=yes or cycleway:right=shared_lane or cycleway:left=track or cycleway:right=track or highway=track and bicycle=designated and motor_vehicle=no
# or highway=path and bicycle=yes or highway=path and bicycle=designated or bicycle=official or highway=service and bicycle=designated or motor_vehicle=no or highway=pedestrian and bicycle=yes 
# or bicycle=official or highway=footway and bicycle=yes or bicycle=official or highway=cycleway 
# or cycleway in lane, opposite_lane, shared_busway, track, opposite_track or cycleway:left in lane, shared_busway or cycleway:right in lane, shared_busway # didn;t include this data
# or cycleway:both=lane or bicycle_road=yes and motor_vehicle=no or bicycle=designated or cyclestreet=yes'

# cycleway=lane or cycleway=opposite_lane or cycleway=shared_busway or cycleway=track or cycleway=opposite_track or cycleway:left=lane or cycleway:left=shared_busway or cycleway:right=lane or cycleway:right=shared_busway

##### Length or count extraction

In [10]:
# function for getting the total count or length of a parameter in a city
def data_extraction(method, city, geometry, filterKey):
    '''Function that creates a dataframe with count of bicycle amenity 
    method - how to agregrate the data (count or length)
    city - city name
    geometry - boundary area from which to pull that data for
    filterKey - how to filter the results; can use multiple attributes; ex geometry type (type:way)
    '''
    URL = 'https://api.ohsome.org/v1/elements/' + method
    data = {"bpolys": geometry, "format": 'csv', "filter": filterKey, "time": "2008-01-01/2023-01-01/P1Y"}
    
    response = requests.post(URL, data=data)
    decoded_content = response.content.decode('utf-8')
    
    # create and format dataframe from csv file
    cr = csv.reader(decoded_content.splitlines(), delimiter=';')
    data_list = list(cr)
    df = pd.DataFrame(data_list[4:], columns=['Date', city])
    df[city] = df[city].astype(float) # convert length column to float
    
    return df

In [15]:
# create a dataframe summing amount of bike lanes for each city for each year from 2008 to 2023

# filter parameters (can remove each clause after and or or, but keep type:way)
filterKey = 'type:way and sidewalk:left:bicycle=yes or cycleway:left=shared_lane or cyclestreet=yes or cycleway:left=shared_busway or cycleway:right=shared_busway or cycleway=shared_busway or cycleway=opposite_lane or highway=bridleway and bicycle=no or highway=track and bicycle=designated and motor_vehicle=no or bicycle=use_sidepath or cycleway=opposite and oneway:bicycle=no or sidewalk:right:bicycle=yes or cycleway:right=shared_lane or cycleway:left=track or cycleway:right=track or highway=track and bicycle=designated and motor_vehicle=no or highway=path and bicycle=yes or highway=path and bicycle=designated or bicycle=official or highway=service and bicycle=designated or motor_vehicle=no or highway=pedestrian and bicycle=yes or bicycle=official or highway=footway and bicycle=yes or bicycle=official or highway=cycleway or cycleway:both=lane or bicycle_road=yes and motor_vehicle=no or bicycle=designated or cyclestreet=yes or cycleway=lane or cycleway=opposite_lane or cycleway=shared_busway or cycleway=track or cycleway=opposite_track or cycleway:left=lane or cycleway:left=shared_busway or cycleway:right=lane or cycleway:right=shared_busway'

city = "London" # this will need to change for each city
df_lanes = data_extraction('length', city, city_geom_df['Geometry String'][0], filterKey) # make dataframe for one city as starter df for merge

# iterate through list of cities
for index, row in city_geom_df.iterrows():
    city = row['Location']
    geom_str = row['Geometry String']
    df = data_extraction('length', city, geom_str, filterKey) # pull data for specific city
    df_lanes = pd.merge(df_lanes, df, how='outer') # merge city data into one dataframe 

df_lanes

,Date,London,LONDON,AMSTERDAM,PARIS,BERLIN,MUNICH,BARCELONA,ROMA,LISBOA
0,2008-01-01T00:00:00Z,232365.55,232365.55,65251.00,25809.43,108543.45,112169.74,0.00,0.00,0.00
1,2009-01-01T00:00:00Z,377486.69,377486.69,140360.77,153607.56,667331.61,512129.27,9229.91,86.65,926.42
2,2010-01-01T00:00:00Z,427144.12,427144.12,264169.20,453945.55,968759.55,726032.55,60292.16,93.03,1166.45
3,2011-01-01T00:00:00Z,523976.53,523976.53,299009.69,555810.42,1165663.87,872933.69,69839.58,2208.73,11769.16
4,2012-01-01T00:00:00Z,634131.46,634131.46,433890.42,623853.94,1402465.38,985310.98,107604.79,2208.06,23948.55
5,2013-01-01T00:00:00Z,791210.80,791210.80,461762.23,715475.20,1606375.62,1027905.61,133082.94,2625.66,65664.24
6,2014-01-01T00:00:00Z,896129.55,896129.55,477522.23,777950.62,1756221.59,1068485.96,144242.06,2627.40,93178.75
7,2015-01-01T00:00:00Z,1022994.10,1022994.10,495341.72,833566.09,1872004.19,1082938.34,232010.37,4234.74,114170.72
8,2016-01-01T00:00:00Z,1085024.90,1085024.90,544876.56,887903.73,1896318.77,1330521.28,244539.81,4234.74,124309.21
9,2017-01-01T00:00:00Z,1151568.19,1151568.19,575079.56,930736.05,1964700.44,1430861.66,257586.72,4333.44,152438.52


In [12]:
# create a dataframe on the total number of bicycle rental locations for each city for each year from 2008 to 2023

# options for filters: (can remove each clause after and, but keep type:node)
filterKey = 'amenity=bicycle_rental and type:node'
# filterKey = 'amenity=bicycle_parking and bicycle_parking=stands and type:node'

city = "London" # this will need to change for each city
df_docks = data_extraction('count', city, city_geom_df['Geometry String'][0], filterKey) # make dataframe for one city as starter df for merge

# iterate through list of cities
for index, row in city_geom_df.iterrows():
    city = row['Location']
    geom_str = row['Geometry String']
    df = data_extraction('count', city, geom_str, filterKey) # pull data for specific city
    df_docks = pd.merge(df_docks, df, how='outer') # merge city data into one dataframe 

df_docks

,Date,London,LONDON,AMSTERDAM,PARIS,BERLIN,MUNICH,BARCELONA,ROMA,LISBOA
0,2008-01-01T00:00:00Z,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009-01-01T00:00:00Z,1.0,1.0,4.0,102.0,1.0,0.0,0.0,0.0,0.0
2,2010-01-01T00:00:00Z,4.0,4.0,6.0,485.0,3.0,2.0,54.0,0.0,0.0
3,2011-01-01T00:00:00Z,333.0,333.0,7.0,690.0,11.0,2.0,59.0,0.0,0.0
4,2012-01-01T00:00:00Z,370.0,370.0,7.0,721.0,46.0,2.0,64.0,0.0,0.0
5,2013-01-01T00:00:00Z,434.0,434.0,9.0,726.0,84.0,4.0,113.0,0.0,0.0
6,2014-01-01T00:00:00Z,473.0,473.0,12.0,752.0,92.0,4.0,139.0,0.0,2.0
7,2015-01-01T00:00:00Z,503.0,503.0,17.0,780.0,115.0,5.0,178.0,0.0,1.0
8,2016-01-01T00:00:00Z,557.0,557.0,24.0,793.0,175.0,12.0,215.0,0.0,1.0
9,2017-01-01T00:00:00Z,591.0,591.0,28.0,788.0,131.0,33.0,248.0,0.0,1.0
